In [1]:
import pandas as pd
from connection import connect


In [ ]:
#connection
co_oltp,etl_conn=connect('production')


## Extract

In [ ]:
#extract DimProductCategory


from deep_translator import GoogleTranslator

category=pd.read_sql_table('product_category',co_oltp,schema='production')


,product_category_alternate_key,english_product_category_name,spanish_product_category_name,french_product_category_name
0,1,Bikes,bicicletas,Vélos
1,2,Components,Componentes,Composants
2,3,Clothing,Ropa,Vêtements
3,4,Accessories,Accesorios,Accessoires


## Transform


In [ ]:
category=category.rename(columns={'product_category_id':'product_category_alternate_key','name':'english_product_category_name'})

category=category.drop(['rowguid','modified_date'],axis=1)
category['spanish_product_category_name']=category['english_product_category_name'].apply(lambda x : GoogleTranslator(source='en',target='es').translate(x) if x is not None else None )
category['french_product_category_name']=category['english_product_category_name'].apply(lambda x : GoogleTranslator(source='en',target='fr').translate(x) if x is not None else None )

category

## Load

In [6]:
category.to_sql('dim_product_category', etl_conn, if_exists='append',index=False)

4